In [12]:
import vertexai
# from vertexai.generative_models import GenerativeModel
from vertexai.generative_models import GenerativeModel, ChatSession, Part, Content
import vertexai.preview.generative_models as generative_models

In [13]:
import os
from dotenv import load_dotenv

load_dotenv()


True

In [14]:
import os
vertexai.init(project="vision-forge-414908", location="us-central1")
#os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")
os.environ['GOOGLE_APPLICATION_CREDENTIALS']="cred.json"

In [15]:
model = GenerativeModel(model_name="gemini-1.5-flash-001",
                             system_instruction="You have to have a conversation with the user about their insurance-related experience. This may be a hospitalization/car accident etc. Ask relevant questions, ask for necessary clarifications, etc. Once you are done with understanding their experience, the user will send you a list of necessary fields they require to fill out a form. You must return a json object with the values for these fields. Ensure that all your information is accurate, elaborate, and professional. This json object will be your final output.",
                            
     )

In [16]:
user_profile = {
  "patient_profile": {
    "personal_details": {
      "name": "Ranjit Sharma",
      "date_of_birth": "1985-06-15",
      "gender": "Male",
      "marital_status": "Married",
      "children": 0,
      "spouse_name": "Anita Sharma",
      "contact_details": {
        "phone": "+91-9876543210",
        "email": "ranjit.sharma@example.com",
        "address": {
          "street": "15, MG Road",
          "city": "Bengaluru",
          "state": "Karnataka",
          "postal_code": "560001",
          "country": "India"
        }
      }
    },
    "insurance_details": {
      "policy_number": "HSI123456789",
      "policy_provider": "SBI Health Insurance",
      "policy_path": "../docs/SBI.pdf",
      "policy_start_date": "2020-01-01",
      "policy_end_date": "2025-01-01",
      "coverage_amount": 500000,
      "premium_amount": 12000,
      "premium_frequency": "Yearly"
    },
    "medical_history": {
      "pre_existing_conditions": [],
      "allergies": [],
      "current_medications": [],
      "past_treatments": []
    },
    "emergency_contact": {
      "name": "Anita Sharma",
      "relationship": "Spouse",
      "phone": "+91-9876543211"
    },
    "claim_history": [
      {
        "claim_id": "CLAIM98765",
        "claim_date": "2023-04-15",
        "hospital_name": "Apollo Hospital",
        "treatment": "Fracture Treatment",
        "claim_amount": 50000,
        "status": "Approved"
      }
    ]
  }
}

In [17]:
chat_history = [
    {
        "role": "user",
        "text": "This is my user profile:\n"+ str(user_profile)
    },
    {
        "role": "model",
        "text": "Okay, got it. Please ask your questions related to insurance."
    },
    
]

In [18]:
history = []
for message in chat_history:
    if message["role"] == "user":
        history.append(
            Content(role="user", parts=[Part.from_text(message["text"])])
        )
    else:
        history.append(
            Content(role="model", parts=[Part.from_text(f'{message["text"]}')])
        )

In [19]:
chat = ChatSession(model=model, history=history)

In [24]:
op = chat.send_message("She was admitted in an in network hospital called Manipal for 3 days from 28/3/2024 - 31/3/2024. She has diabetes. We had a total of 32 thousand rupees in doctor fees. ")

In [25]:
op.candidates[0].content.parts[0].text

"Thank you for providing that information. It's helpful to know about your wife's diabetes. It's likely that this is a pre-existing condition that will need to be considered when processing your claim.  \n\nTo give you a clear idea of your options, I'd need a bit more information. Could you please tell me:\n\n* **What type of insurance policy do you have?** Is it a health insurance policy, a hospitalization policy, or something else?\n* **Does your policy specifically cover dengue?** \n* **What is your policy's maximum coverage limit for a single hospitalization?** \n\nOnce I have this information, I can assess whether your claim is likely to be approved and what documentation you'll need to gather. \n"

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoModel
model_name = "jinaai/jina-embeddings-v2-base-en"
model_kwargs ={'trust_remote_code': True}


c:\Users\Aakash Rajaraman\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
embeddings_model = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-small-en",model_kwargs=model_kwargs)


In [2]:
from werkzeug.datastructures import ImmutableMultiDict

In [3]:
x = ImmutableMultiDict([('policy_number', ''), ('policyholder_name', ''), ('dob', ''), ('contact_number', ''), ('email', ''), ('address', ''), ('claim_amount', ''), ('date_of_treatment', ''), ('hospital_name', ''), ('hospital_address', ''), ('diagnosis', ''), ('treatment_description', ''), ('doctor_name', ''), ('doctor_contact', ''), ('policy_start_date', ''), ('policy_end_date', ''), ('has_previous_claims', 'yes'), ('supporting_documents', '')])

In [6]:
x.keys()

dict_keys(['policy_number', 'policyholder_name', 'dob', 'contact_number', 'email', 'address', 'claim_amount', 'date_of_treatment', 'hospital_name', 'hospital_address', 'diagnosis', 'treatment_description', 'doctor_name', 'doctor_contact', 'policy_start_date', 'policy_end_date', 'has_previous_claims', 'supporting_documents'])

In [1]:
import os
import google.generativeai as genai
from google.generativeai import caching
import datetime
import time
import dotenv
from vertexai.generative_models import Part

c:\Users\Aakash Rajaraman\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']="cred.json"

In [9]:
path = r"../docs/health_form.pdf"

In [14]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        extracted_text = ""
        for page in pdf_reader.pages:
            text = page.extract_text()
            if text:
                print("t", text)
                extracted_text += text
        return extracted_text


In [15]:
print(extract_text_from_pdf(path))

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [4]:
paths =[
    "../docs/car insurance.pdf",
    "../docs/united india health insurance.pdf",
    "../docs/home insurance.pdf",
]

In [5]:
docs = [PyPDFLoader(url).load() for url in paths]
docs_list = [item for sublist in docs for item in sublist]

In [6]:

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(docs_list)

In [7]:
ids = ["id"+str(i) for i in range(1,len(doc_splits)+1)]
print(ids)

['id1', 'id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8', 'id9', 'id10', 'id11', 'id12', 'id13', 'id14', 'id15', 'id16', 'id17', 'id18', 'id19', 'id20', 'id21', 'id22', 'id23', 'id24', 'id25', 'id26', 'id27', 'id28', 'id29', 'id30', 'id31', 'id32', 'id33', 'id34', 'id35', 'id36', 'id37', 'id38', 'id39', 'id40', 'id41', 'id42', 'id43', 'id44', 'id45', 'id46', 'id47', 'id48', 'id49', 'id50', 'id51', 'id52', 'id53', 'id54', 'id55', 'id56', 'id57', 'id58', 'id59', 'id60', 'id61', 'id62', 'id63', 'id64', 'id65', 'id66', 'id67', 'id68', 'id69', 'id70', 'id71', 'id72', 'id73', 'id74', 'id75', 'id76', 'id77', 'id78', 'id79', 'id80', 'id81', 'id82', 'id83', 'id84', 'id85', 'id86', 'id87', 'id88', 'id89', 'id90', 'id91', 'id92', 'id93', 'id94', 'id95', 'id96', 'id97', 'id98', 'id99', 'id100', 'id101', 'id102', 'id103', 'id104', 'id105', 'id106', 'id107', 'id108', 'id109', 'id110', 'id111', 'id112', 'id113', 'id114', 'id115', 'id116', 'id117', 'id118', 'id119', 'id120', 'id121', 'id122', 'id123', 

In [8]:
final_doc_splits = [i.page_content for i in doc_splits]

In [9]:
import chromadb
from langchain_chroma import Chroma
persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("collection_name")
collection.add(ids=ids, documents=final_doc_splits)

vector_store_from_client = Chroma(
    client=persistent_client,
    collection_name="collection_name",
    embedding_function=embeddings_model,
)

c:\Users\Aakash Rajaraman\AppData\Local\Programs\Python\Python311\Lib\site-packages\onnxruntime\capi\_pybind_state.py:26: UserWarning: Please install the 2019 Visual C++ runtime and then try again. If you've installed the runtime in a non-standard location (other than %SystemRoot%\System32), make sure it can be found by setting the correct path.
  warnings.warn("Please install the 2019 Visual C++ runtime and then try again. "


: 

135

In [48]:
for i in doc_splits:
    
    if type(i.page_content) == str:
        print("no worries")
    else:
        print("Not a string")

no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries
no worries